In [81]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats

### Functions to calculate queueing model inputs & steps

In [215]:
def calculate_stdev_coeffvar_serverate(df) :
    """Calculate key inputs for queueing model: Service Rate, and St Dev / Coefficient of Variation of service time"""
    df['Time_Mean_hrs'] = df['Time_Mean'] / 60
    df['Time_WorstCase_hrs'] = df['Time_WorstCase'] / 60
    df['Time_StDev'] = df['Time_WorstCase_hrs'] / stats.norm.ppf(1 - df['Perc_WorstCase'])
    df['Time_CoeffVarS'] = df['Time_StDev'] / df['Time_Mean_hrs']
    df['ServiceRate'] = 1/ df['Time_Mean_hrs']
    return df

In [195]:
def load_factor_to_utilization(df, process) :
    """Calculate load factor on step in queueing model, and return utilization (max 1)"""
    series = df.loc[df['Process'] == process]
    load_factor = series['ArrivalRate'].item() / (series['ServiceRate'].item() * series['Servers'].item())
    if load_factor > 1 :
        load_factor = 1
    return load_factor

In [196]:
def wait_in_queue(df, process) :
    """Calculate wait in queue"""
    series = df.loc[df['Process'] == process]
    clock_speed = 1 / (series['ServiceRate'].item() * series['Servers'].item())
    explosion_effect = 1 / (1 - series['Utilization'].item())
    scale_effect = utilization ** ((np.sqrt((series['Servers'].item() + 1) * 2)) - 1)
    variability_effect = (series['Time_CoeffVarA'].item() ** 2 + series['Time_CoeffVarS'].item() ** 2) / 2
    wait_in_queue = clock_speed * explosion_effect * scale_effect * variability_effect
    return wait_in_queue

### Data tables

Available providers (first level of resource)

In [197]:
providers = {'Doctor': 9, 'Nurse': 5, 'FlowStaff': 20, 'CSR': 10}
providers

{'Doctor': 9, 'Nurse': 5, 'FlowStaff': 20, 'CSR': 10}

Variability - % of time (based on dad's estimates) that worst case occurs

In [198]:
low = 0.05
medium = 0.1
high = 0.2

Base case arrival rate

In [199]:
arrival_rate = 100 / 10
arrival_std = 5 / 60 #Std of arrivals of 5 minutes
arrivals_coeff_var = arrival_std / arrival_rate
arrivals = [arrival_rate, arrival_std, arrivals_coeff_var]
arrivals

[10.0, 0.08333333333333333, 0.008333333333333333]

Base case breakdown of care types and service time scenarios

In [200]:
cols = ['Type', 'Frequency', 'Time_Mean', 'Time_WorstCase', 'Perc_WorstCase']
preventative = ['Preventative', 0.2, 30, 30, low]
chronic = ['Chronic', 0.6, 15, 5, medium]
acute = ['Acute', 0.2, 30, 30, low]
base_case_types = pd.DataFrame([preventative, chronic, acute], columns = cols)

In [201]:
# Create blended type that is weighted average of current case types
blended_mean = np.average(base_case_types['Time_Mean'], weights = base_case_types['Frequency'])
blended_worstcase = np.average(base_case_types['Time_WorstCase'], weights = base_case_types['Frequency'])
blended_perc = np.average(base_case_types['Perc_WorstCase'], weights = base_case_types['Frequency'])
blended = ['Blended', 1, blended_mean, blended_worstcase, blended_perc]
base_case_types.loc[len(base_case_types)] = blended

In [202]:
base_case_types = calculate_stdev_coeffvar_serverate(base_case_types)

In [203]:
base_case_types

,Type,Frequency,Time_Mean,Time_WorstCase,Perc_WorstCase,Time_Mean_hrs,Time_WorstCase_hrs,Time_StDev,Time_CoeffVarS,ServiceRate
0,Preventative,0.2,30.0,30.0,0.05,0.50,0.500000,0.303978,0.607957,2.000000
1,Chronic,0.6,15.0,5.0,0.10,0.25,0.083333,0.065025,0.260101,4.000000
2,Acute,0.2,30.0,30.0,0.05,0.50,0.500000,0.303978,0.607957,2.000000
3,Blended,1.0,21.0,15.0,0.08,0.35,0.250000,0.177927,0.508363,2.857143


Base case flow process - _Exam_ step to be modified based on type of case (from table above), using `Blended` for now

In [204]:
cols = ['Step', 'Process', 'Staff', 'Time_Mean', 'Time_WorstCase', 'Perc_WorstCase']
appt = ['Schedule', 'Make_appointment', 'CSR', 5, 5, low]
checkin = ['Arrive', 'Check_in', 'CSR', 2, 3, low]
wait = ['Arrive', 'Waiting_room', 'FlowStaff', 3, 5, low] # ADJUSTED ON OWN -- WAIT TIME TO COME FROM QUEUEING BUILDUP
to_room = ['Arrive', 'To_exam_room', 'FlowStaff', 1, 0, low]
vitals = ['Exam_prep', 'Vitals_check', 'FlowStaff', 2, 0, low]
refine_complaint = ['Exam_prep', 'Refine_complaint', 'FlowStaff', 15, 15, high]
start_note = ['Exam_prep', 'Start_note', 'FlowStaff', 1, 0, low]
exam = ['Exam_provider', 'Exam', 'Doctor', base_case_types.loc[base_case_types['Type'] == 'Blended', 'Time_Mean'].item(), base_case_types.loc[base_case_types['Type'] == 'Blended', 'Time_WorstCase'].item(), base_case_types.loc[base_case_types['Type'] == 'Blended', 'Perc_WorstCase'].item()]
checkout = ['Conclude', 'Checkout', 'CSR', 5, 5, medium] # ADJUSTED ON OWN -- WAIT TIME TO COME FROM QUEUEING BUILDUP
process_flow = pd.DataFrame([appt, checkin, wait, to_room, vitals, refine_complaint, start_note, exam, checkout], columns = cols)

In [205]:
process_flow = calculate_stdev_coeffvar_serverate(process_flow)

In [206]:
process_flow['Servers'] = process_flow['Staff'].map(providers)

In [207]:
process_flow

,Step,Process,Staff,Time_Mean,Time_WorstCase,Perc_WorstCase,Time_Mean_hrs,Time_WorstCase_hrs,Time_StDev,Time_CoeffVarS,ServiceRate,Servers
0,Schedule,Make_appointment,CSR,5.0,5.0,0.05,0.083333,0.083333,0.050663,0.607957,12.000000,10
1,Arrive,Check_in,CSR,2.0,3.0,0.05,0.033333,0.050000,0.030398,0.911935,30.000000,10
2,Arrive,Waiting_room,FlowStaff,3.0,5.0,0.05,0.050000,0.083333,0.050663,1.013261,20.000000,20
3,Arrive,To_exam_room,FlowStaff,1.0,0.0,0.05,0.016667,0.000000,0.000000,0.000000,60.000000,20
4,Exam_prep,Vitals_check,FlowStaff,2.0,0.0,0.05,0.033333,0.000000,0.000000,0.000000,30.000000,20
5,Exam_prep,Refine_complaint,FlowStaff,15.0,15.0,0.20,0.250000,0.250000,0.297046,1.188183,4.000000,20
6,Exam_prep,Start_note,FlowStaff,1.0,0.0,0.05,0.016667,0.000000,0.000000,0.000000,60.000000,20
7,Exam_provider,Exam,Doctor,21.0,15.0,0.08,0.350000,0.250000,0.177927,0.508363,2.857143,9
8,Conclude,Checkout,CSR,5.0,5.0,0.10,0.083333,0.083333,0.065025,0.780304,12.000000,10


### Basic queueing model

Start after `Make_appointment` process

In [247]:
process_flow_clinic = process_flow.iloc[1:]

First round set manually with `ArrivalRate` pulled in from estimated data.

In [248]:
process_flow_clinic.loc[1, 'ArrivalRate'] = arrivals[0]
process_flow_clinic.loc[1, 'Time_CoeffVarA'] = arrivals[2]

In [249]:
process_flow_clinic

,Step,Process,Staff,Time_Mean,Time_WorstCase,Perc_WorstCase,Time_Mean_hrs,Time_WorstCase_hrs,Time_StDev,Time_CoeffVarS,ServiceRate,Servers,ArrivalRate,Time_CoeffVarA
1,Arrive,Check_in,CSR,2.0,3.0,0.05,0.033333,0.050000,0.030398,0.911935,30.000000,10,10.0,0.008333
2,Arrive,Waiting_room,FlowStaff,3.0,5.0,0.05,0.050000,0.083333,0.050663,1.013261,20.000000,20,NaN,NaN
3,Arrive,To_exam_room,FlowStaff,1.0,0.0,0.05,0.016667,0.000000,0.000000,0.000000,60.000000,20,NaN,NaN
4,Exam_prep,Vitals_check,FlowStaff,2.0,0.0,0.05,0.033333,0.000000,0.000000,0.000000,30.000000,20,NaN,NaN
5,Exam_prep,Refine_complaint,FlowStaff,15.0,15.0,0.20,0.250000,0.250000,0.297046,1.188183,4.000000,20,NaN,NaN
6,Exam_prep,Start_note,FlowStaff,1.0,0.0,0.05,0.016667,0.000000,0.000000,0.000000,60.000000,20,NaN,NaN
7,Exam_provider,Exam,Doctor,21.0,15.0,0.08,0.350000,0.250000,0.177927,0.508363,2.857143,9,NaN,NaN
8,Conclude,Checkout,CSR,5.0,5.0,0.10,0.083333,0.083333,0.065025,0.780304,12.000000,10,NaN,NaN


In [250]:
utilization = load_factor_to_utilization(process_flow_clinic, 'Check_in')
process_flow_clinic.loc[1, 'Utilization'] = utilization

In [251]:
wiq = wait_in_queue(process_flow_clinic, 'Check_in')
process_flow_clinic.loc[1, 'WaitInQueue'] = wiq

In [252]:
process_flow_clinic.loc[1, 'TimeInSystem'] = process_flow_clinic.loc[1, 'WaitInQueue'] + process_flow_clinic.loc[1, 'Time_Mean_hrs']

In [253]:
process_flow_clinic.loc[1, 'PotentialOutput'] = 1 / process_flow_clinic.loc[1, 'TimeInSystem']

In [254]:
process_flow_clinic

,Step,Process,Staff,Time_Mean,Time_WorstCase,Perc_WorstCase,Time_Mean_hrs,Time_WorstCase_hrs,Time_StDev,Time_CoeffVarS,ServiceRate,Servers,ArrivalRate,Time_CoeffVarA,Utilization,WaitInQueue,TimeInSystem,PotentialOutput
1,Arrive,Check_in,CSR,2.0,3.0,0.05,0.033333,0.050000,0.030398,0.911935,30.000000,10,10.0,0.008333,0.033333,5.073933e-09,0.033333,29.999995
2,Arrive,Waiting_room,FlowStaff,3.0,5.0,0.05,0.050000,0.083333,0.050663,1.013261,20.000000,20,NaN,NaN,NaN,NaN,NaN,NaN
3,Arrive,To_exam_room,FlowStaff,1.0,0.0,0.05,0.016667,0.000000,0.000000,0.000000,60.000000,20,NaN,NaN,NaN,NaN,NaN,NaN
4,Exam_prep,Vitals_check,FlowStaff,2.0,0.0,0.05,0.033333,0.000000,0.000000,0.000000,30.000000,20,NaN,NaN,NaN,NaN,NaN,NaN
5,Exam_prep,Refine_complaint,FlowStaff,15.0,15.0,0.20,0.250000,0.250000,0.297046,1.188183,4.000000,20,NaN,NaN,NaN,NaN,NaN,NaN
6,Exam_prep,Start_note,FlowStaff,1.0,0.0,0.05,0.016667,0.000000,0.000000,0.000000,60.000000,20,NaN,NaN,NaN,NaN,NaN,NaN
7,Exam_provider,Exam,Doctor,21.0,15.0,0.08,0.350000,0.250000,0.177927,0.508363,2.857143,9,NaN,NaN,NaN,NaN,NaN,NaN
8,Conclude,Checkout,CSR,5.0,5.0,0.10,0.083333,0.083333,0.065025,0.780304,12.000000,10,NaN,NaN,NaN,NaN,NaN,NaN


Subsequent processes using `ArrivalRate` as minimum of input passed through and potential output of step.

In [255]:
for i in process_flow_clinic[process_flow_clinic['ArrivalRate'].isnull()].index :
    process = process_flow_clinic.loc[i, 'Process']
    arrival = min(process_flow_clinic.loc[i - 1, 'ArrivalRate'], process_flow_clinic.loc[i - 1, 'PotentialOutput'])
    process_flow_clinic.loc[i, 'ArrivalRate'] = arrival
    process_flow_clinic.loc[i, 'Time_CoeffVarA'] = 0
    process_flow_clinic.loc[i, 'Utilization'] = load_factor_to_utilization(process_flow_clinic, process)
    process_flow_clinic.loc[i, 'WaitInQueue'] = wait_in_queue(process_flow_clinic, process)
    process_flow_clinic.loc[i, 'TimeInSystem'] = process_flow_clinic.loc[i, 'WaitInQueue'] + process_flow_clinic.loc[i, 'Time_Mean_hrs']
    process_flow_clinic.loc[i, 'PotentialOutput'] = 1 / process_flow_clinic.loc[i, 'TimeInSystem']

_Output times of process flow through the system_

In [256]:
process_flow_clinic[['Process', 'ArrivalRate', 'Utilization', 'WaitInQueue', 'TimeInSystem']]

,Process,ArrivalRate,Utilization,WaitInQueue,TimeInSystem
1,Check_in,10.000000,0.033333,5.073933e-09,0.033333
2,Waiting_room,10.000000,0.025000,1.055917e-11,0.050000
3,To_exam_room,10.000000,0.008333,0.000000e+00,0.016667
4,Vitals_check,10.000000,0.016667,0.000000e+00,0.033333
5,Refine_complaint,10.000000,0.125000,8.089469e-11,0.250000
6,Start_note,4.000000,0.003333,0.000000e+00,0.016667
7,Exam,4.000000,0.155556,4.423904e-08,0.350000
8,Checkout,2.857142,0.023810,9.195820e-09,0.083333


In [259]:
hrs = process_flow_clinic['TimeInSystem'].sum()
minutes = hrs * 60
print ('Total time in system: {:.2f} hrs // {:.2f} min'.format(hrs, minutes))

Total time in system: 0.83 hrs // 50.00 min


In [261]:
print ('Mean processing time (estimated): {:.2f} min'.format(process_flow_clinic['Time_Mean'].sum()))

Mean processing time (estimated): 50.00 min
